In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import rs_access_v1 as rs

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure plotting style for professional output
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("🚀 Environment setup complete!")
print("📊 Ready to analyze fare brand structures")


In [ ]:
def get_us_domestic_data():
    """
    Fetch US domestic flight data from Redshift using metadata.airportlocation_extra
    """
    print("📊 Fetching US domestic flight data from Redshift...")
    
    # Initialize Redshift connection
    rs.assign_connection("ds")
    
    # Query for US domestic markets using the airport location table
    query = """
    SELECT 
        carrier,
        origin,
        destination,
        outbound_departure_date,
        outbound_fare_family,
        inbound_fare_family,
        price_inc,
        price_exc,
        tax,
        yqyr,
        currency,
        search_class,
        cabin,
        outbound_cabins,
        inbound_cabins,
        outbound_booking_class,
        inbound_booking_class,
        outbound_fare_basis,
        inbound_fare_basis,
        refundable,
        change_fee,
        los,
        sales_date,
        observation_date,
        customer,
        outbound_travel_stop_over,
        inbound_travel_stop_over
    FROM common_output.common_output_format 
    WHERE sales_date = 20250629
    AND origin IN (SELECT airportcode FROM metadata.airportlocation_extra WHERE countryname = 'United States')
    AND destination IN (SELECT airportcode FROM metadata.airportlocation_extra WHERE countryname = 'United States')
    AND origin != destination
    AND carrier IS NOT NULL
    AND outbound_fare_family IS NOT NULL
    AND price_inc > 0
    LIMIT 75000;
    """
    
    df = rs.rq(query)
    
    if df is not None and len(df) > 0:
        print(f"✅ Fetched {len(df):,} records for US domestic markets")
        return df
    else:
        print("❌ No data returned from query")
        return pd.DataFrame()

# Execute data fetch
df_raw = get_us_domestic_data()


In [ ]:
def clean_and_prepare_data(df):
    """
    Clean and prepare the data for analysis
    """
    print("🧹 Cleaning and preparing data...")
    
    if df.empty:
        print("❌ No data to clean")
        return df
    
    # Create copy to avoid modifying original
    df = df.copy()
    
    # Convert date columns
    df['outbound_departure_date'] = pd.to_datetime(df['outbound_departure_date'], errors='coerce')
    df['observation_date'] = pd.to_datetime(df['observation_date'], errors='coerce')
    df['sales_date'] = pd.to_datetime(df['sales_date'], format='%Y%m%d', errors='coerce')
    
    # Calculate days to departure (advance purchase window)
    df['days_to_departure'] = (df['outbound_departure_date'] - df['observation_date']).dt.days
    
    # Create market identifier
    df['market'] = df['origin'] + '-' + df['destination']
    
    # Handle missing fare families
    df['outbound_fare_family'] = df['outbound_fare_family'].fillna('Unknown')
    df['inbound_fare_family'] = df['inbound_fare_family'].fillna('Unknown')
    
    # Create primary fare family (outbound for analysis)
    df['primary_fare_family'] = df['outbound_fare_family']
    
    # Flag round-trip vs one-way
    df['is_roundtrip'] = df['inbound_fare_family'].notna() & (df['inbound_fare_family'] != 'Unknown')
    
    # Clean price data
    df['price_inc'] = pd.to_numeric(df['price_inc'], errors='coerce')
    df['price_exc'] = pd.to_numeric(df['price_exc'], errors='coerce')
    df['change_fee'] = pd.to_numeric(df['change_fee'], errors='coerce')
    
    # Remove invalid data
    initial_count = len(df)
    df = df.dropna(subset=['carrier', 'market', 'primary_fare_family', 'price_inc'])
    df = df[df['price_inc'] > 0]  # Remove zero/negative prices
    
    # Only keep records with valid advance purchase data
    df = df.dropna(subset=['days_to_departure'])
    df = df[df['days_to_departure'] >= 0]  # Remove past departures
    
    # Define advance purchase buckets as specified in requirements
    def categorize_advance_purchase(days):
        """Categorize advance purchase days into buckets"""
        if days <= 7:
            return '0-7 days'
        elif days <= 14:
            return '8-14 days'
        elif days <= 21:
            return '15-21 days'
        elif days <= 30:
            return '22-30 days'
        else:
            return '30+ days'
    
    df['advance_purchase_bucket'] = df['days_to_departure'].apply(categorize_advance_purchase)
    
    print(f"✅ Cleaned data: {len(df):,} records (removed {initial_count - len(df):,} invalid records)")
    
    if len(df) > 0:
        print(f"📅 Date range: {df['outbound_departure_date'].min()} to {df['outbound_departure_date'].max()}")
        print(f"📊 Advance purchase range: {df['days_to_departure'].min()} to {df['days_to_departure'].max()} days")
        print(f"✈️ Airlines: {sorted(df['carrier'].unique())}")
        print(f"🗺️ Markets: {df['market'].nunique()}")
    
    return df

# Clean the data
df_clean = clean_and_prepare_data(df_raw)


In [ ]:
# Perform comprehensive exploratory data analysis
print("=" * 60)
print("EXPLORATORY DATA ANALYSIS")
print("=" * 60)

if not df_clean.empty:
    # Basic statistics
    print(f"\n📈 Dataset Overview:")
    print(f"Total records: {len(df_clean):,}")
    print(f"Unique airlines: {df_clean['carrier'].nunique()}")
    print(f"Unique markets: {df_clean['market'].nunique()}")
    print(f"Unique fare families: {df_clean['primary_fare_family'].nunique()}")
    
    # Airlines distribution
    print(f"\n✈️ Airlines in dataset:")
    airline_counts = df_clean['carrier'].value_counts()
    for airline, count in airline_counts.head(10).items():
        print(f"  {airline}: {count:,} records")
    
    # Market distribution
    print(f"\n🗺️ Top markets:")
    market_counts = df_clean['market'].value_counts()
    for market, count in market_counts.head(10).items():
        print(f"  {market}: {count:,} records")
    
    # Fare family distribution
    print(f"\n🏷️ Fare families overview:")
    ff_counts = df_clean['primary_fare_family'].value_counts()
    for ff, count in ff_counts.head(15).items():
        print(f"  {ff}: {count:,} records")
    
    # Price statistics
    print(f"\n💰 Price statistics:")
    print(f"  Mean price: ${df_clean['price_inc'].mean():.2f}")
    print(f"  Median price: ${df_clean['price_inc'].median():.2f}")
    print(f"  Price range: ${df_clean['price_inc'].min():.2f} - ${df_clean['price_inc'].max():.2f}")
    
    # Advance purchase statistics
    print(f"\n📅 Advance purchase statistics:")
    ap_counts = df_clean['advance_purchase_bucket'].value_counts()
    for bucket, count in ap_counts.items():
        pct = (count / len(df_clean)) * 100
        print(f"  {bucket}: {count:,} records ({pct:.1f}%)")
else:
    print("❌ No data for EDA")


In [ ]:
# Analyze brand availability by advance purchase window
print("=" * 60)
print("ADVANCE PURCHASE PATTERN ANALYSIS")
print("=" * 60)

if not df_clean.empty:
    # Analyze brand availability by advance purchase window
    ap_analysis = df_clean.groupby(['carrier', 'market', 'advance_purchase_bucket', 'primary_fare_family']).agg({
        'price_inc': ['count', 'mean', 'min'],
        'refundable': 'mean',
        'change_fee': 'mean'
    }).reset_index()
    
    # Flatten column names
    ap_analysis.columns = ['carrier', 'market', 'advance_purchase_bucket', 'fare_family',
                          'record_count', 'avg_price', 'min_price', 'refundable_pct', 'avg_change_fee']
    
    print(f"\n📅 Brand availability by advance purchase window:")
    
    # Show top brands by advance purchase window
    for bucket in ['0-7 days', '8-14 days', '15-21 days']:
        print(f"\n{bucket}:")
        bucket_data = ap_analysis[ap_analysis['advance_purchase_bucket'] == bucket].groupby('fare_family').agg({
            'record_count': 'sum',
            'avg_price': 'mean'
        }).reset_index().sort_values('record_count', ascending=False)
        
        for _, row in bucket_data.head(8).iterrows():
            print(f"  {row['fare_family']}: {row['record_count']:,} records, avg ${row['avg_price']:.2f}")
    
    print(f"\n✅ Advance purchase analysis complete!")
else:
    print("❌ No data for advance purchase analysis")
    ap_analysis = pd.DataFrame()


In [ ]:
# Basic Economy Identification
print("=" * 60)
print("BASIC ECONOMY IDENTIFICATION")
print("=" * 60)

if not df_clean.empty:
    # Create comprehensive analysis for each airline-market-brand combination
    brand_metrics = df_clean.groupby(['carrier', 'market', 'primary_fare_family']).agg({
        'price_inc': ['count', 'mean', 'min', 'max', 'std'],
        'refundable': 'mean',
        'change_fee': ['mean', 'max'],
        'days_to_departure': 'mean',
        'outbound_booking_class': lambda x: len(set(x))  # number of booking classes
    }).reset_index()
    
    # Flatten column names
    brand_metrics.columns = ['carrier', 'market', 'fare_family', 'record_count', 
                           'avg_price', 'min_price', 'max_price', 'price_std',
                           'refundable_pct', 'avg_change_fee', 'max_change_fee',
                           'avg_days_out', 'booking_class_variety']
    
    # Calculate price rank within each airline-market combination
    brand_metrics['price_rank'] = brand_metrics.groupby(['carrier', 'market'])['avg_price'].rank(method='min')
    
    # Basic Economy scoring function
    def calculate_basic_economy_score(row):
        """Calculate Basic Economy score based on multiple factors"""
        score = 0
        
        # Price criteria (40% weight)
        if row['price_rank'] == 1:  # Cheapest average price
            score += 40
        elif row['price_rank'] == 2:
            score += 30
        elif row['price_rank'] <= 3:
            score += 20
        
        # Refundability (20% weight)
        if pd.notna(row['refundable_pct']):
            if row['refundable_pct'] < 0.1:  # Less than 10% refundable
                score += 20
            elif row['refundable_pct'] < 0.3:
                score += 10
        
        # Change fee criteria (20% weight)
        if pd.notna(row['avg_change_fee']):
            if row['avg_change_fee'] > 100:  # High change fees
                score += 20
            elif row['avg_change_fee'] > 50:
                score += 10
        
        # Brand name analysis (20% weight)
        fare_family_lower = str(row['fare_family']).lower()
        basic_keywords = ['basic', 'economy', 'main', 'standard', 'saver', 'light', 'essential']
        premium_keywords = ['first', 'business', 'premium', 'plus', 'comfort', 'extra', 'flex']
        
        if any(keyword in fare_family_lower for keyword in basic_keywords):
            score += 15
        if any(keyword in fare_family_lower for keyword in premium_keywords):
            score -= 15
        
        return score
    
    # Apply scoring function
    brand_metrics['basic_economy_score'] = brand_metrics.apply(calculate_basic_economy_score, axis=1)
    
    # Identify Basic Economy for each airline-market combination
    basic_economy_candidates = brand_metrics.loc[
        brand_metrics.groupby(['carrier', 'market'])['basic_economy_score'].idxmax()
    ].copy()
    
    # Add confidence level
    def assign_confidence(score):
        if score >= 70:
            return 'High'
        elif score >= 50:
            return 'Medium'
        else:
            return 'Low'
    
    basic_economy_candidates['confidence'] = basic_economy_candidates['basic_economy_score'].apply(assign_confidence)
    
    print(f"\n🎯 Basic Economy Identification Results:")
    print(f"Total airline-market combinations analyzed: {len(basic_economy_candidates):,}")
    
    confidence_dist = basic_economy_candidates['confidence'].value_counts()
    for conf, count in confidence_dist.items():
        print(f"  {conf} confidence: {count:,} combinations")
    
    # Show examples by airline
    print(f"\n✈️ Basic Economy candidates by airline:")
    for carrier in basic_economy_candidates['carrier'].unique()[:8]:
        carrier_data = basic_economy_candidates[basic_economy_candidates['carrier'] == carrier]
        high_conf = carrier_data[carrier_data['confidence'] == 'High']
        if len(high_conf) > 0:
            most_common_brand = high_conf['fare_family'].mode()
            if len(most_common_brand) > 0:
                print(f"  {carrier}: {most_common_brand[0]} (in {len(high_conf)} markets)")
    
    print(f"\n✅ Basic Economy identification complete!")
else:
    print("❌ No data for Basic Economy identification")
    basic_economy_candidates = pd.DataFrame()
    brand_metrics = pd.DataFrame()


In [ ]:
# Create comprehensive visualizations
print("=" * 60)
print("CREATING VISUALIZATIONS")
print("=" * 60)

if not df_clean.empty:
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Fare Brand Analysis and Basic Economy Detection - US Domestic Markets', 
                 fontsize=16, fontweight='bold')
    
    # 1. Airline distribution
    airline_counts = df_clean['carrier'].value_counts().head(10)
    axes[0, 0].bar(airline_counts.index, airline_counts.values, color='skyblue')
    axes[0, 0].set_title('Top Airlines by Record Count')
    axes[0, 0].set_xlabel('Airline')
    axes[0, 0].set_ylabel('Number of Records')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Price distribution by airline
    top_airlines = df_clean['carrier'].value_counts().head(6).index
    price_data = df_clean[df_clean['carrier'].isin(top_airlines)]
    price_data.boxplot(column='price_inc', by='carrier', ax=axes[0, 1])
    axes[0, 1].set_title('Price Distribution by Airline')
    axes[0, 1].set_xlabel('Airline')
    axes[0, 1].set_ylabel('Price (USD)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Advance purchase distribution
    ap_counts = df_clean['advance_purchase_bucket'].value_counts()
    axes[0, 2].pie(ap_counts.values, labels=ap_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 2].set_title('Distribution by Advance Purchase Window')
    
    # 4. Brand availability by advance purchase window
    if not ap_analysis.empty:
        ap_summary = ap_analysis.groupby(['advance_purchase_bucket', 'fare_family']).agg({
            'record_count': 'sum'
        }).reset_index()
        
        # Get top 5 brands for visualization
        top_brands = ap_summary.groupby('fare_family')['record_count'].sum().nlargest(5).index
        
        ap_pivot = ap_summary[ap_summary['fare_family'].isin(top_brands)].pivot(
            index='advance_purchase_bucket', columns='fare_family', values='record_count'
        ).fillna(0)
        
        ap_pivot.plot(kind='bar', ax=axes[1, 0], width=0.8)
        axes[1, 0].set_title('Brand Availability by Advance Purchase Window')
        axes[1, 0].set_xlabel('Advance Purchase Window')
        axes[1, 0].set_ylabel('Number of Records')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 5. Basic Economy confidence distribution
    if not basic_economy_candidates.empty:
        conf_counts = basic_economy_candidates['confidence'].value_counts()
        colors = ['#2E8B57', '#FFA500', '#DC143C']  # Green, Orange, Red
        axes[1, 1].bar(conf_counts.index, conf_counts.values, color=colors)
        axes[1, 1].set_title('Basic Economy Detection Confidence')
        axes[1, 1].set_xlabel('Confidence Level')
        axes[1, 1].set_ylabel('Number of Combinations')
        
        # Add percentage labels
        total = conf_counts.sum()
        for i, (conf, count) in enumerate(conf_counts.items()):
            pct = (count / total) * 100
            axes[1, 1].text(i, count + 5, f'{pct:.1f}%', ha='center', va='bottom')
    
    # 6. Basic Economy score distribution
    if not basic_economy_candidates.empty:
        axes[1, 2].hist(basic_economy_candidates['basic_economy_score'], bins=20, color='salmon', alpha=0.7)
        axes[1, 2].set_title('Basic Economy Score Distribution')
        axes[1, 2].set_xlabel('Basic Economy Score')
        axes[1, 2].set_ylabel('Frequency')
        axes[1, 2].axvline(x=70, color='red', linestyle='--', label='High Confidence Threshold')
        axes[1, 2].axvline(x=50, color='orange', linestyle='--', label='Medium Confidence Threshold')
        axes[1, 2].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizations created successfully!")
else:
    print("❌ No data for visualizations")


In [ ]:
# Create the final deliverable table
print("=" * 60)
print("CREATING DELIVERABLE TABLE")
print("=" * 60)

if not basic_economy_candidates.empty and not brand_metrics.empty:
    # Group by airline and market to get all brands
    brand_analysis = df_clean.groupby(['carrier', 'market']).agg({
        'primary_fare_family': lambda x: ', '.join(sorted(list(set(x)))),
        'price_inc': ['count', 'mean', 'min', 'max'],
        'days_to_departure': ['mean', 'min', 'max']
    }).reset_index()
    
    # Flatten column names
    brand_analysis.columns = ['carrier', 'market', 'all_brands', 'record_count', 
                             'avg_price', 'min_price', 'max_price', 
                             'avg_days_out', 'min_days_out', 'max_days_out']
    
    # Merge with basic economy identification
    deliverable = brand_analysis.merge(
        basic_economy_candidates[['carrier', 'market', 'fare_family', 'basic_economy_score', 'confidence']],
        on=['carrier', 'market'],
        how='left'
    )
    
    # Create final deliverable table
    deliverable_final = deliverable[['carrier', 'market', 'all_brands', 'fare_family', 'confidence', 'basic_economy_score']].copy()
    deliverable_final.columns = ['Airline', 'Market', 'All_Detected_Brands', 'Identified_Basic_Economy_Brand', 'Confidence_Level', 'BE_Score']
    
    # Sort by airline and market
    deliverable_final = deliverable_final.sort_values(['Airline', 'Market'])
    
    print(f"📋 Deliverable table created with {len(deliverable_final):,} rows")
    print(f"\nSample of deliverable table:")
    print(deliverable_final.head(10).to_string(index=False))
    
    # Save to CSV
    output_filename = f"output/basic_economy_analysis_enhanced_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    deliverable_final.to_csv(output_filename, index=False)
    print(f"\n💾 Saved deliverable table to: {output_filename}")
    
    # Show summary statistics
    print(f"\n📊 Summary Statistics:")
    print(f"Total airline-market combinations: {len(deliverable_final):,}")
    print(f"Unique airlines: {deliverable_final['Airline'].nunique()}")
    print(f"Unique markets: {deliverable_final['Market'].nunique()}")
    
    confidence_summary = deliverable_final['Confidence_Level'].value_counts()
    print(f"\nConfidence distribution:")
    for conf, count in confidence_summary.items():
        pct = (count / len(deliverable_final)) * 100
        print(f"  {conf}: {count:,} combinations ({pct:.1f}%)")
    
else:
    print("❌ No data for deliverable table")
    deliverable_final = pd.DataFrame()
